In [19]:
import pandas as pd
import requests
import json
from config import gkey

In [20]:
# Create dataframe from cleaned data
clean_data = pd.read_csv("Resources/drug_death_data_clean.csv")
clean_data.head()

,Unnamed: 0,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,...,Benzodiazepine,Methadone,Amphetamine,Tramadol,Morphine Not Heroin,Hydromorphone,Other,OpiateNOS,Any Opioid,MannerofDeath
0,1,13-0102,03/21/2013 12:00:00 AM,DateofDeath,48.0,Male,Black,NORWALK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident
1,2,16-0165,03/13/2016 12:00:00 AM,DateofDeath,30.0,Female,White,SANDY HOOK,FAIRFIELD,CT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Accident
2,3,16-0208,03/31/2016 12:00:00 AM,DateofDeath,23.0,Male,White,RYE,WESTCHESTER,NY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Accident
3,4,13-0052,02/13/2013 12:00:00 AM,DateofDeath,22.0,Male,"Asian, Other",FLUSHING,QUEENS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident
4,5,14-0277,06/29/2014 12:00:00 AM,DateofDeath,23.0,Male,White,BRISTOL,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident


In [21]:
# Group data by Death City by count
city_grouped = clean_data.groupby("DeathCity")
city_counts = pd.DataFrame(city_grouped.count()["ID"])
city_counts = city_counts.reset_index()
city_counts = city_counts.rename(columns={"DeathCity":"City","ID":"Deaths"})
# Drop city 06340
city_counts = city_counts.loc[city_counts["City"] != "06340",:]

#city_counts

In [22]:
# Create new columns for city coordinates
city_counts["Latitude"] = ""
city_counts["Longitude"] = ""


# Call google places api for each city and retrieve coordinates
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
params = {"key":gkey,
         "input":"",
         "inputtype":"textquery",
         "fields":"geometry/location"}

for index, row in city_counts.iterrows():
    # Change input for each row/city.  Add "CT" since we're working in Connecticut
    params["input"] = row["City"] + ",CT"
    # Call google places API
    response = requests.get(base_url,params)
    print(f"Processing {row['City']}...")
    # If good response, store lat and lng in df
    if response.status_code == 200:
        try:
            city_counts.loc[index,"Latitude"] = response.json()['candidates'][0]["geometry"]["location"]["lat"]
            city_counts.loc[index,"Longitude"] = response.json()['candidates'][0]["geometry"]["location"]["lng"]
        except IndexError:
          print(f"Information not found for {row['City']}! Continuing...")
    else:
        print(f"Information not found for {row['City']}! Continuing...")
              
print("Latitude/Longitude processing complete!")

Processing AMSTON...
Processing ANDOVER...
Processing ANSONIA...
Processing ASHFORD...
Processing AVON...
Processing BAKERSVILLE...
Processing BALTIC...
Processing BANTAM...
Processing BARKHAMSTED...
Processing BEACON FALLS...
Processing BERLIN...
Processing BETHANY...
Processing BETHEL...
Processing BETHLEHEM...
Processing BLOOMFIELD...
Processing BOLTON...
Processing BOZRAH...
Processing BRANFORD...
Processing BRIDGEPORT...
Processing BRIDGEWATER...
Processing BRISTOL...
Processing BROAD BROOK...
Processing BROOKFIELD...
Processing BROOKLYN...
Processing BURLINGTON...
Processing CANTERBURY...
Processing CANTON...
Processing CENTRAL VILLAGE...
Processing CHAPLIN...
Processing CHESHIRE...
Processing CHESTER...
Processing CLINTON...
Processing COLCHESTER...
Processing COLUMBIA...
Processing CORNWALL BRIDGE...
Processing COS COB...
Processing COVENTRY...
Processing CROMWELL...
Processing DANBURY...
Processing DANIELSON...
Processing DARIEN...
Processing DAYVILLE...
Processing DEEP RIVER.

In [23]:
# Verify lat/lng has been added.
city_counts

,City,Deaths,Latitude,Longitude
1,AMSTON,1,41.6254,-72.3431
2,ANDOVER,3,41.7373,-72.3704
3,ANSONIA,23,41.3462,-73.079
4,ASHFORD,6,41.8731,-72.1216
5,AVON,5,41.8096,-72.8305
6,BAKERSVILLE,1,41.832,-73.0301
7,BALTIC,3,41.6172,-72.0851
8,BANTAM,1,41.7245,-73.2362
9,BARKHAMSTED,1,41.9293,-72.914
10,BEACON FALLS,4,41.4426,-73.0618


In [24]:
# Group data by Death County by count
#clean_data.loc[clean_data["DeathCounty"]=="USA",:]
# One of the counties is "USA"; it's city is HARTFORD.  Change county to HARTFORD
clean_data["DeathCounty"] = clean_data["DeathCounty"].replace({"USA":"HARTFORD"})
# Clean any/all whitespaces around county names
clean_data["DeathCounty"] = clean_data["DeathCounty"].str.strip()

# Now that the data is cleaner, group it
county_grouped = clean_data.groupby("DeathCounty")
county_counts = pd.DataFrame(county_grouped.count()["ID"])
county_counts = county_counts.rename_axis('County')
county_counts = county_counts.rename(columns={"ID":"Deaths"})

county_counts

,Deaths
County,
FAIRFIELD,597
HARTFORD,1187
LITCHFIELD,235
MIDDLESEX,177
NEW HAVEN,1068
NEW LONDON,357
TOLLAND,109
WINDHAM,134


In [32]:
# Want to center the map on CT, so we need coordinates for the "center" of CT.
ct_coord = (41.5032, -72.6877)

In [33]:
# Configure gmaps
import gmaps
gmaps.configure(api_key=gkey)

In [34]:
# Load geojson_geometries for US Counties
import gmaps.geojson_geometries
county_geojson = gmaps.geojson_geometries.load_geometry('us-counties')


In [35]:
# Create geo_json of just CT
ct_geojson = {'type':'FeatureCollection',
             'features':[]}

for county in county_geojson['features']:
    if county['properties']["STATE"] == '09':
        ct_geojson['features'].append(county)

# Print list of counties for order
counties_ordered = []
for county in ct_geojson['features']:
    counties_ordered.append(county['properties']['NAME'])

counties_ordered

['Windham',
 'Tolland',
 'Hartford',
 'Middlesex',
 'Litchfield',
 'Fairfield',
 'New London',
 'New Haven']

In [36]:
# To convert deaths/weights to color, I'm modifying a color normalizer from the geojson documentation:
# https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#geojson-layer
# Requires a min and max value which I've manually pulled from the county data:
min_deaths = 100
max_deaths = 1000 # Setting max intensity to 400 for better vis
death_range = max_deaths - min_deaths
# Import neccessary functionality:
from matplotlib.cm import inferno
from matplotlib.colors import to_hex

def calculate_color(weight):
    """
    Convert the weight to a color
    """
    # make weight a number between 0 and 1
    normalized_deaths = (weight - min_deaths) / death_range

    # invert deaths so that high inequality gives dark color
    inverse_deaths = 1.0 - normalized_deaths

    # transform the deaths/weights to a matplotlib color
    mpl_color = inferno(inverse_deaths)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

In [37]:
# Add column with color values
county_counts['Color'] = ''
for county,deaths in county_counts.iterrows():
    color = calculate_color(deaths["Deaths"])
    county_counts.loc[county,'Color'] = color
    
# Add colors to list in correct order
colors = []
for county in counties_ordered:
    colors.append(county_counts.loc[county.upper(),'Color'])
    
colors

['#f2f482',
 '#f9fc9d',
 '#000004',
 '#f4df53',
 '#fbbe23',
 '#a62d60',
 '#f57d15',
 '#000004']

In [38]:
# Deaths by County Map
#figure_layout_0 = {}
county_map = gmaps.figure(center=ct_coord, zoom_level=8.4)
geo_layer = gmaps.geojson_layer(ct_geojson,
                                fill_color=colors,
#                                stroke_color=colors,
                                fill_opacity=0.6)
county_map.add_layer(geo_layer)

county_map

Figure(layout=FigureLayout(height='420px'))

In [39]:
# Deaths by City Map

#figure_layout_1 = {backgroundColor:'gray'}
city_map = gmaps.figure(center=ct_coord, zoom_level=8.4)
heatmap_layer = gmaps.heatmap_layer(
    city_counts[["Latitude","Longitude"]],
    weights=city_counts["Deaths"],
    max_intensity=100,
    point_radius=40.0,
    gradient=((0,0,0,0.2),'yellow',(255,160,0,0.8),'red')
    )
city_map.add_layer(heatmap_layer)
city_map

Figure(layout=FigureLayout(height='420px'))